# HW 1

**Fall 2025 Quantiative Methods in Finance**

**The Erdös Institute**

1) Consider the process $X$ of randomly selecting numbers from the unit interval $[0,1]$ until the sum of values exceeds $1$. Prove that $E[X]=e$.

2) Consider the random variable $X$ that selects a number from the two element $\{1,-1\}$ with the uniform distribution until the sum of the drawn values is $1$. Prove that $E[X] = \infty$.

3) Show the probability of winning a pass-line bet in Craps (Vegas rules) is $\frac{244}{495}$.

4) A gambler has \$256 to gamble with and plays a doubling strategy in a game with a 45% chance of winning and a 1:1 payout:

- They bet \$2. If they win, they stop playing.
- If they lose, they double their previous bet and try again.
- This continues until they either win, in which case they take their profits and walk away, or lose the \$128 bet. At that point their bankroll is exhausted.


a) Find the expected profit of the gambler. 

b) Use Monte-Carlo methods to estimate the expected profit of the gambler.

c) What is the probability the strategy is profitable?

d) Use Monte-Carlo methods to estimate the probability the strategy is profitable.